In [1]:
#imports
import pickle
import torch
import math
import numpy as np
from collections import Counter

In [2]:
#project specific imports
from modules.utils import *
from modules.build_dataset import *

In [3]:
#OP_tsv: OpenProt 1.6
#Ens_trx: mart_export (ensembl 95)
#trx_fasta: transcripts fasta (ensembl 95)
#sorfs: list of ribo-seq orfs (mudge & al., 2022)
data = Data(
    OP_tsv = 'data/human-openprot-r1_6-refprots+altprots+isoforms-+uniprot2019_03_01.tsv', 
    Ens_trx = 'data/mart_export.txt',
    trx_fasta = 'data/ensembl95_trxps.fasta',
    sorfs = 'data/sorfs.xlsx',
    unique_pept = 'data/alts_TE3_UniqPep2_MS3.csv'
)

In [4]:
#ensembl_trx = data.ensembl_trx()
#pickle.dump(ensembl_trx, open('data/ensembl_trx.pkl', 'wb'))
ensembl_trx = pickle.load(open('data/ensembl_trx.pkl', 'rb'))

In [5]:
#trx_orfs = data.trx_orfs(ensembl_trx)
#pickle.dump(trx_orfs, open('data/trx_orfs.pkl', 'wb'))
trx_orfs = pickle.load(open('data/trx_orfs.pkl', 'rb'))

In [219]:
seq1 = torch.rand(1, 5)
seq2 = torch.rand(1, 6)
seq3 = torch.rand(1, 7)
seq4 = torch.rand(1, 11)

In [259]:
seq1 = torch.rand(4, 5)
seq2 = torch.rand(4, 6)
seq3 = torch.rand(4, 7)
seq4 = torch.rand(4, 11)

In [260]:
seqs = [seq1, seq2, seq3, seq4]

In [276]:
def pad_seqs(seqs, num_chan):
    pad_seqs = []
    max_len = max([x.shape[1] for x in seqs])
    for seq in seqs:
        diff_len = max_len - seq.shape[1]
        padL, padR = torch.zeros(num_chan, diff_len//2), torch.zeros(num_chan, diff_len//2+diff_len%2)
        pad_seq = torch.cat([padL, seq, padR], dim=1)
        pad_seqs.append(pad_seq)
    return torch.stack(pad_seqs, dim=0)

In [277]:
padded_seqs = pad_seqs(seqs, 4)

In [288]:
torch.stack([x[0] != 0 for x in padded_seqs])

tensor([[False, False, False,  True,  True,  True,  True,  True, False, False,
         False],
        [False, False,  True,  True,  True,  True,  True,  True, False, False,
         False],
        [False, False,  True,  True,  True,  True,  True,  True,  True, False,
         False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True]])

In [279]:
padded_seqs != 0

tensor([[[False, False, False,  True,  True,  True,  True,  True, False, False,
          False],
         [False, False, False,  True,  True,  True,  True,  True, False, False,
          False],
         [False, False, False,  True,  True,  True,  True,  True, False, False,
          False],
         [False, False, False,  True,  True,  True,  True,  True, False, False,
          False]],

        [[False, False,  True,  True,  True,  True,  True,  True, False, False,
          False],
         [False, False,  True,  True,  True,  True,  True,  True, False, False,
          False],
         [False, False,  True,  True,  True,  True,  True,  True, False, False,
          False],
         [False, False,  True,  True,  True,  True,  True,  True, False, False,
          False]],

        [[False, False,  True,  True,  True,  True,  True,  True,  True, False,
          False],
         [False, False,  True,  True,  True,  True,  True,  True,  True, False,
          False],
         [False,

In [263]:
def utility_fct(Xy):
    X, y = Xy
    X, y = pad_seqs(X, 4), pad_seqs(y, 1)
    return (X, y)

In [ ]:
dataset = data.dataset(ensembl_trx, trx_orfs)
pickle.dump(dataset, open('data/dataset.pkl', 'wb'))
#dataset = pickle.load(open('data/dataset.pkl', 'rb'))

In [ ]:
from sklearn.model_selection import train_test_split
X = [x['mapped_seq'] for x in dataset.values()]
y = [x['mapped_cds'] for x in dataset.values()]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

train_split = (X_train,y_train)
test_split = (X_test,y_test)
pickle.dump(train_split, open('data/train_split.pkl', 'wb'))
pickle.dump(test_split, open('data/test_split.pkl', 'wb'))